# 実行環境の作成

## Google Cloud Storage

下記コードでGCPに接続

In [38]:
from google.colab import auth
auth.authenticate_user()

認証に成功したらgcsfuseをインストール

In [ ]:
!echo "deb http://packages.cloud.google.com/apt gcsfuse-`lsb_release -c -s` main" | sudo tee /etc/apt/sources.list.d/gcsfuse.list
!curl https://packages.cloud.google.com/apt/doc/apt-key.gpg | sudo apt-key add -
!apt-get -y -q update
!apt-get -y -q install gcsfuse

バケット「statistics-hyogo」をディレクトリ「statistics-hyogo」にマウント

In [ ]:
! mkdir -p statistics-hyogo
! gcsfuse --implicit-dirs --limit-bytes-per-sec -1 --limit-ops-per-sec -1 statistics-hyogo statistics-hyogo

## ESTAT_APP_ID

In [6]:
ESTAT_APPID = '724e5b90772a3e9289f41a253e4e7e32438f4fff'

## pythonのライブラリ追加

In [7]:
import urllib.parse
import urllib.request
import json
import pandas as pd

# estatの統計情報を取得

'statsDataId'に統計表コードを指定

In [8]:
statsDataId = '0000010101'

メタ情報を取得する関数を作成

In [ ]:
def get_estat_meta(statsDataId):
  # リクエストパラメータ
  p={}
  p['statsDataId'] = statsDataId
  p['appId'] = ESTAT_APPID

  url = 'http://api.e-stat.go.jp/rest/3.0/app/json/getMetaInfo?'
  url += urllib.parse.urlencode(p)

  with urllib.request.urlopen(url) as response:
    return json.loads(response.read().decode('utf-8'))

meta = get_estat_meta(statsDataId)
# print(meta)

## DataFrameの作成

メタ情報をDataFrameに整形する

In [31]:
def get_cards_dataframe(statsDataId):
  # メタ情報の取得
  meta= get_estat_meta(statsDataId)
 
  # STAT_NAME
  STAT_NAME = meta['GET_META_INFO']['METADATA_INF']['TABLE_INF']['STAT_NAME']['$']

  # CLASS_OBJ
  CLASS_OBJ = meta['GET_META_INFO']['METADATA_INF']['CLASS_INF']['CLASS_OBJ']

  # cat01
  cat01 = next((d for d in CLASS_OBJ if d['@id'] == 'cat01'), None)['CLASS']

  # DataFrame
  df = pd.json_normalize(cat01)[['@code','@name']]
  
  # 統計表情報の追加
  df['statsDataId'] = statsDataId
  df['statsDataName'] = STAT_NAME

  # 列名の変更
  columns = {'@code':'categoryCode','@name':'categoryName'}
  df= df.rename(columns=columns)

  # categoryNameから不要な情報（categoryCode）を削除
  df['categoryName'] = df.apply(lambda x: x['categoryName'].replace(x['categoryCode']+'_', ''), 1)
  
  
  # 並べ替え
  df = df.reindex(columns=['statsDataId', 'statsDataName', 'categoryCode','categoryName'])
  
  return df


# print(len(category_codes))

In [32]:
df = get_cards_dataframe(statsDataId)
df

,statsDataId,statsDataName,categoryCode,categoryName
0,0000010101,社会・人口統計体系,A1101,総人口
1,0000010101,社会・人口統計体系,A110101,総人口（男）
2,0000010101,社会・人口統計体系,A110102,総人口（女）
3,0000010101,社会・人口統計体系,A1102,日本人人口
4,0000010101,社会・人口統計体系,A110201,日本人人口（男）
...,...,...,...,...
563,0000010101,社会・人口統計体系,A850205,父子世帯数（55歳以上）
564,0000010101,社会・人口統計体系,A9101,婚姻件数
565,0000010101,社会・人口統計体系,A9111,平均婚姻年齢（初婚の夫）
566,0000010101,社会・人口統計体系,A9112,平均婚姻年齢（初婚の妻）


## 統計カード管理情報をSpreadSheetで加工

統計カードの管理情報を SpreadSheetで管理する

前セルで作業したDataFrameをcsvでコピーするなどして作業すること


[statistics-hyogo/統計カード管理](https://docs.google.com/spreadsheets/d/1mAv2gx9khNOty_ILu3aJjJz0uoRkARzwoaG4VBUSNG4/edit#gid=0)

## BigQueryへ保存

ライブラリのインストール

In [36]:
!pip install --upgrade -q gspread

SpreadSheetの認証

In [40]:
from google.colab import auth
from google.auth import default
import gspread

auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

worksheetを読み込んでDataFrameに変換

In [53]:
# スプレッドシートの情報
ss_id = "1mAv2gx9khNOty_ILu3aJjJz0uoRkARzwoaG4VBUSNG4"
workbook = gc.open_by_key(ss_id)
worksheet = workbook.worksheet("シート1")

# DataFrameに格納
dic = worksheet.get_all_records(head=1)
df = pd.DataFrame(dic)
df

,index,statsDataId,statsDataName,categoryCode,categoryName,governmentType,cardId,cardTitle,chartComponent,isSelect,type,yAxis
0,0,10101,社会・人口統計体系,A1101,総人口,prefecture,total-population-prefecture,都道府県の総人口,TimeChart,TRUE,column,0
1,1,10101,社会・人口統計体系,A110101,総人口（男）,prefecture,total-population-prefecture,都道府県の総人口,TimeChart,,column,0
2,2,10101,社会・人口統計体系,A110102,総人口（女）,prefecture,total-population-prefecture,都道府県の総人口,TimeChart,,column,0
3,3,10101,社会・人口統計体系,A1102,日本人人口,prefecture,,,,,,
4,4,10101,社会・人口統計体系,A110201,日本人人口（男）,prefecture,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...
563,563,10101,社会・人口統計体系,A850205,父子世帯数（55歳以上）,prefecture,,,,,,
564,564,10101,社会・人口統計体系,A9101,婚姻件数,prefecture,,,,,,
565,565,10101,社会・人口統計体系,A9111,平均婚姻年齢（初婚の夫）,prefecture,first-marriage-age-prefecture,都道府県の平均初婚年齢,TimeChart,TRUE,line,0
566,566,10101,社会・人口統計体系,A9112,平均婚姻年齢（初婚の妻）,prefecture,first-marriage-age-prefecture,都道府県の平均初婚年齢,TimeChart,TRUE,line,0


pandas-gbqのインストール

In [ ]:
!pip install pandas-gbq

DataFrameをBigQueryへ保存

In [54]:
import pandas as pd

# プロジェクトの定義
project_id='primal-buttress-342908'
dataset_id = 'contents'
table_id='cards'

# データフレームの内容をBigQueryのテーブルに追加
df.to_gbq( f'{dataset_id}.{table_id}', project_id=project_id, if_exists="replace")

1it [00:04,  4.92s/it]


# 統計カード管理情報からデータセット

BigQueryから

## estat-APIのデータ取得

In [ ]:
params = {}

params['statsDataId'] = statsDataId
params['cdCat01'] = 'A1231'
params['cdArea']=",".join(area_codes)

# print (params)

estat-APIのデータを取得する関数を作成

In [ ]:
def get_estat_data(params):
  p = params.copy()

  # appId
  p['appId'] = ESTAT_APPID
  
  # url生成
  url = 'http://api.e-stat.go.jp/rest/2.1/app/json/getStatsData?'
  url += urllib.parse.urlencode(p)

  with urllib.request.urlopen(url) as response:
    return json.loads(response.read().decode('utf-8'))

res = get_estat_data(params)
# print(res)

## pandas DataFrameに変換

estat-APIのレスポンスからVALUEを抜き出し、DataFrameに変換する

In [ ]:
res = get_estat_data(params)

# VALUE
VALUE = res['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE']

df_value = pd.json_normalize(VALUE)
df_value

,@tab,@cat01,@area,@time,@unit,$
0,00001,A1231,01000,2010100000,歳,47.7
1,00001,A1231,01000,2015100000,歳,49.7
2,00001,A1231,02000,2010100000,歳,49.1
3,00001,A1231,02000,2015100000,歳,51.4
4,00001,A1231,03000,2010100000,歳,49.5
...,...,...,...,...,...,...
89,00001,A1231,45000,2015100000,歳,50.2
90,00001,A1231,46000,2010100000,歳,49.0
91,00001,A1231,46000,2015100000,歳,50.6
92,00001,A1231,47000,2010100000,歳,40.0


カテゴリ情報は'CLASS_OBJ'に格納されている。

これもDataFrameに変換

In [ ]:
# CLASS_INF
CLASS_OBJ = res['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']

categories = next((d for d in CLASS_OBJ if d['@id'] == 'cat01'), None)['CLASS']

df_categories = pd.json_normalize(categories)
df_categories

統計表情報は'TABLE_INF'に格納されている

In [ ]:
# TABLE_INF
TABLE_INF = res['GET_STATS_DATA']['STATISTICAL_DATA']['TABLE_INF']

statDataId = TABLE_INF['@id']
statDataName = TABLE_INF['STAT_NAME']['$']

print(statDataId,statDataName)

estat-APIのレスポンスをひとつのDataFrameにまとめる関数を作成

In [ ]:
def conv_estat_response_to_dataframe(response):
  # CLASS_INF
  CLASS_OBJ = response['GET_STATS_DATA']['STATISTICAL_DATA']['CLASS_INF']['CLASS_OBJ']

  # TABLE_INF
  TABLE_INF = response['GET_STATS_DATA']['STATISTICAL_DATA']['TABLE_INF']

  # VALUE
  VALUE = response['GET_STATS_DATA']['STATISTICAL_DATA']['DATA_INF']['VALUE']

  # VALUEをDataFrameに変換
  df_res = pd.json_normalize(VALUE)

  # CLASS_OBJのDataFrameを結合
  for d in CLASS_OBJ :
    # DataFrameに変換 '@code','@name'だけ抽出
    df_class = pd.json_normalize(d['CLASS']) 
    df_class = df_class.copy()[['@code','@name']]

    # @codeをキー名に変更 @nameをキー名＋'_name'に変更
    key_name = '@{}'.format(d['@id'])
    df_class = df_class.rename(columns={'@code':key_name,'@name': key_name+'_name'})

    # DataFrameを結合
    df_res = pd.merge(df_res, df_class, on=key_name, how='outer')

  # 統計情報を追加
  df_res['statsDataId'] = TABLE_INF['@id']
  df_res['statsDataName'] = TABLE_INF['STAT_NAME']['$']
  
  return df_res

df_org = conv_estat_response_to_dataframe(res)
df_org

,@tab,@cat01,@area,@time,@unit,$,@tab_name,@cat01_name,@area_name,@time_name,statsDataId,statsDataName
0,00001,A1231,01000,2010100000,歳,47.7,観測値,A1231_年齢中位数,北海道,2010年度,0000010101,社会・人口統計体系
1,00001,A1231,02000,2010100000,歳,49.1,観測値,A1231_年齢中位数,青森県,2010年度,0000010101,社会・人口統計体系
2,00001,A1231,03000,2010100000,歳,49.5,観測値,A1231_年齢中位数,岩手県,2010年度,0000010101,社会・人口統計体系
3,00001,A1231,04000,2010100000,歳,44.8,観測値,A1231_年齢中位数,宮城県,2010年度,0000010101,社会・人口統計体系
4,00001,A1231,05000,2010100000,歳,52.4,観測値,A1231_年齢中位数,秋田県,2010年度,0000010101,社会・人口統計体系
...,...,...,...,...,...,...,...,...,...,...,...,...
133,NaN,NaN,NaN,2016100000,NaN,NaN,NaN,NaN,NaN,2016年度,0000010101,社会・人口統計体系
134,NaN,NaN,NaN,2017100000,NaN,NaN,NaN,NaN,NaN,2017年度,0000010101,社会・人口統計体系
135,NaN,NaN,NaN,2018100000,NaN,NaN,NaN,NaN,NaN,2018年度,0000010101,社会・人口統計体系
136,NaN,NaN,NaN,2019100000,NaN,NaN,NaN,NaN,NaN,2019年度,0000010101,社会・人口統計体系


## DataFrameの整形

使い勝手が良くなるように、DataFrameを整形していく

必要な列だけ抽出する

In [ ]:
df_1 = df_org[['statsDataId','statsDataName','@cat01','@cat01_name','@time','@time_name','@area','@area_name','$','@unit']]

# print(df_1)

列名を変更する

In [ ]:
columns = {'@cat01':'categoryCode','@cat01_name':'categoryName','@time':'timeCode','@time_name':'timeName','@area':'areaCode','@area_name':'areaName','$':'value','@unit':'unit'}
df_1 = df_1.rename(columns=columns)

# print(df_1)

欠損値が含まれる行を削除する

In [ ]:
df_1 = df_1.dropna()

# print(df_1)

categoryNameから不要な情報（categoryCode）を削除する

In [ ]:
df_1['categoryName'] = df_1.apply(lambda x: x['categoryName'].replace(x['categoryCode']+'_', ''), 1)

# print(df_1)

timeCodeを4桁文字列に置換する

In [ ]:
df_1['timeCode'] = df_1.apply(lambda x: x['timeCode'][:4], 1)

# print(df_1)

ここまでを関数にまとめる

In [ ]:
def  format_estat_dataframe(df_arg):
  # 必要な列だけ抽出
  df_res =  df_arg[['statsDataId','statsDataName','@cat01','@cat01_name','@time','@time_name','@area','@area_name','$','@unit']]

  # 列名の変更
  columns = {'@cat01':'categoryCode','@cat01_name':'categoryName','@time':'timeCode','@time_name':'timeName','@area':'areaCode','@area_name':'areaName','$':'value','@unit':'unit'}
  df_res = df_res.rename(columns=columns)

  # 欠損データ削除
  df_res = df_res.dropna()

  # categoryNameから不要な情報（categoryCode）を削除
  df_res['categoryName'] = df_res.apply(lambda x: x['categoryName'].replace(x['categoryCode']+'_', ''), 1)

  # timeCodeを文字列4桁に置換
  df_res['timeCode'] = df_res.apply(lambda x: x['timeCode'][:4], 1)

  return df_res

df_fmt = format_estat_dataframe(df_org)
df_fmt

,statsDataId,statsDataName,categoryCode,categoryName,timeCode,timeName,areaCode,areaName,value,unit
0,0000010101,社会・人口統計体系,A1231,年齢中位数,2010,2010年度,01000,北海道,47.7,歳
1,0000010101,社会・人口統計体系,A1231,年齢中位数,2010,2010年度,02000,青森県,49.1,歳
2,0000010101,社会・人口統計体系,A1231,年齢中位数,2010,2010年度,03000,岩手県,49.5,歳
3,0000010101,社会・人口統計体系,A1231,年齢中位数,2010,2010年度,04000,宮城県,44.8,歳
4,0000010101,社会・人口統計体系,A1231,年齢中位数,2010,2010年度,05000,秋田県,52.4,歳
...,...,...,...,...,...,...,...,...,...,...
89,0000010101,社会・人口統計体系,A1231,年齢中位数,2015,2015年度,43000,熊本県,49.0,歳
90,0000010101,社会・人口統計体系,A1231,年齢中位数,2015,2015年度,44000,大分県,50.2,歳
91,0000010101,社会・人口統計体系,A1231,年齢中位数,2015,2015年度,45000,宮崎県,50.2,歳
92,0000010101,社会・人口統計体系,A1231,年齢中位数,2015,2015年度,46000,鹿児島県,50.6,歳


## 順位を付与する

年度ごとに順位を付与

In [ ]:
def format_estat_dataframe_withrank(df_arg):

  # 年度リストを作成
  times = df_arg['timeCode'].tolist()
  times = list(set(times))

  # 返却するDataFrameの定義
  df_res = pd.DataFrame(index=[], columns=[])

  # 年度毎に順位を付与
  for time in times:

    # 単年度のdataを抽出
    df_time = df_arg.copy()[df_arg['timeCode'] == time]

    # valueを数値に変換して、'rank'を付与
    df_time = df_time.astype({'value': float})
    df_time['rank'] = df_time.rank(ascending=False)['value'].astype(int).astype(str)

    # 結合
    df_res = pd.concat([df_res, df_time])
    df_res = df_res.astype({'value': str})

  return df_res

df_last = format_estat_dataframe_withrank(df_fmt)
df_last

,statsDataId,statsDataName,categoryCode,categoryName,timeCode,timeName,areaCode,areaName,value,unit,rank
47,0000010101,社会・人口統計体系,A1231,年齢中位数,2015,2015年度,01000,北海道,49.7,歳,18
48,0000010101,社会・人口統計体系,A1231,年齢中位数,2015,2015年度,02000,青森県,51.4,歳,5
49,0000010101,社会・人口統計体系,A1231,年齢中位数,2015,2015年度,03000,岩手県,51.3,歳,6
50,0000010101,社会・人口統計体系,A1231,年齢中位数,2015,2015年度,04000,宮城県,46.7,歳,37
51,0000010101,社会・人口統計体系,A1231,年齢中位数,2015,2015年度,05000,秋田県,54.8,歳,1
...,...,...,...,...,...,...,...,...,...,...,...
42,0000010101,社会・人口統計体系,A1231,年齢中位数,2010,2010年度,43000,熊本県,47.8,歳,19
43,0000010101,社会・人口統計体系,A1231,年齢中位数,2010,2010年度,44000,大分県,48.8,歳,13
44,0000010101,社会・人口統計体系,A1231,年齢中位数,2010,2010年度,45000,宮崎県,48.6,歳,15
45,0000010101,社会・人口統計体系,A1231,年齢中位数,2010,2010年度,46000,鹿児島県,49.0,歳,10


## まとめ

paramsを指定してから、順位を付与したDataFrameを取得するまでを関数にまとめる

In [ ]:
params = {}

params['statsDataId'] = statsDataId
params['cdCat01'] = 'A1101'
params['cdArea']=",".join(area_codes)

In [ ]:
def get_estat_dataframe(params):
  # estat-APIのレスポンス取得
  res = get_estat_data(params)

  # レスポンスをDataFrameに変換
  df = conv_estat_response_to_dataframe(res)

  # DataFrameを整形
  df = format_estat_dataframe(df)

  # 順位を付与
  df = format_estat_dataframe_withrank(df)

  return df

df = get_estat_dataframe(params)
df

,statsDataId,statsDataName,categoryCode,categoryName,timeCode,timeName,areaCode,areaName,value,unit,rank
1504,0000010101,社会・人口統計体系,A1101,総人口,2007,2007年度,01000,北海道,5579000.0,人,8
1505,0000010101,社会・人口統計体系,A1101,総人口,2007,2007年度,02000,青森県,1409000.0,人,29
1506,0000010101,社会・人口統計体系,A1101,総人口,2007,2007年度,03000,岩手県,1364000.0,人,32
1507,0000010101,社会・人口統計体系,A1101,総人口,2007,2007年度,04000,宮城県,2354000.0,人,15
1508,0000010101,社会・人口統計体系,A1101,総人口,2007,2007年度,05000,秋田県,1121000.0,人,37
...,...,...,...,...,...,...,...,...,...,...,...
700,0000010101,社会・人口統計体系,A1101,総人口,1989,1989年度,43000,熊本県,1841000.0,人,22
701,0000010101,社会・人口統計体系,A1101,総人口,1989,1989年度,44000,大分県,1240000.0,人,32
702,0000010101,社会・人口統計体系,A1101,総人口,1989,1989年度,45000,宮崎県,1172000.0,人,36
703,0000010101,社会・人口統計体系,A1101,総人口,1989,1989年度,46000,鹿児島県,1805000.0,人,23


In [ ]:
pip install pandas_gbq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# プロジェクトの定義
project_id='primal-buttress-342908'
dataset_id = 'primal-buttress-342908.esat_api'
table_id='test3'


df = get_estat_dataframe(params)
# データフレームの内容をBigQueryのテーブルに追加
df.to_gbq( f'{dataset_id}.{table_id}', project_id=project_id, if_exists="append")

GenericGBQException: ignored

# 統計表に含まれる全てのデータをDataframeに変換する

'statsDataId'で統計表を指定して、含まれる統計データを一括でDataFrameに変換する

In [ ]:
statsDataId = '0000010101'

In [ ]:
def get_estat_dataframe_all(statsDataId):

  # 統計表のメタ情報を取得
  meta = get_estat_meta(statsDataId)

  # カテゴリ一覧
  category_codes = get_estat_catefory_codes(statsDataId)

  # 地域コード一覧
  area_codes = get_estat_area_codes(statsDataId)

  # 返却するDataFrameの定義
  df_res = pd.DataFrame(index=[], columns=[])

  for category_code in category_codes:
    # リクエストパラメータ生成
    params = {}
    params['statsDataId'] = statsDataId
    params['cdCat01'] = category_code
    params['cdArea']=",".join(area_codes)

    # DataFrameを取得
    print(category_code)
    df_category = get_estat_dataframe(params)

    df_res = pd.concat([df_res, df_category])

  # return df_res
  return 'test'


df = get_estat_dataframe_all(statsDataId)
# df


# 新しいセクション